# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

C. Fendt  ->  C. Fendt  |  ['C. Fendt']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
I. Hammond  ->  I. Hammond  |  ['I. Hammond']
D. Trevascus  ->  D. Trevascus  |  ['D. Trevascus']
M. Samland  ->  M. Samland  |  ['M. Samland']
D. Trevascus  ->  D. Trevascus  |  ['D. Trevascus']
E. Matthews  ->  E. Matthews  |  ['E. Matthews']
I. Hammond  ->  I. Hammond  |  ['I. Hammond']
W. Brandner  ->  W. Brandner  |  ['W. Brandner']
R. Burn  ->  R. Burn  |  ['R. Burn']
G. Chauvin  ->  G. Chauvin  |  ['G. Chauvin']
L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
J. Sauter  ->  J. Sauter  |  ['J. Sauter']
G. Perotti  ->  G. Perotti  |  ['G. Perotti']
T. Henning  ->  T. Henning  |  ['T. Henning']
K. Schwarz  ->  K. Schwarz  |  ['K. Schwarz']
R. v. Boekel  ->  R. v. Boekel  |  ['R. v. Boekel']
T. Henning  ->  T. Henning  |  ['T. Henning']
G. Perotti  ->  G. Perotti  |  ['G. Perotti']
K. Schwarz  ->  K. Schwarz  |  ['K. Schwarz']
Arxiv has 67 new papers today
       

# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/6 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2504.10599


extracting tarball to tmp_2504.10599...

 done.
Retrieving document from  https://arxiv.org/e-print/2504.10931


/tmp/ipykernel_3240/2822249172.py:52: LatexWarning: 2504.10931 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


...

Found 139 bibliographic references in tmp_2504.11424/sample631.bbl.
Issues with the citations
syntax error in line 450: '}' expected
Retrieving document from  https://arxiv.org/e-print/2504.11425


extracting tarball to tmp_2504.11425...

 done.


T. Henning  ->  T. Henning  |  ['T. Henning']
G. Perotti  ->  G. Perotti  |  ['G. Perotti']
K. Schwarz  ->  K. Schwarz  |  ['K. Schwarz']


Found 51 bibliographic references in tmp_2504.11425/sample631.bbl.
Issues with the citations
syntax error in line 25: '=' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.11127-b31b1b.svg)](https://arxiv.org/abs/2504.11127) | **Keplerian motion of a compact source orbiting the inner disk of PDS 70: A third protoplanet in a resonance with b and c?**  |
|| <mark>I. Hammond</mark>, et al. -- incl., <mark>D. Trevascus</mark>, <mark>M. Samland</mark> |
|*Appeared on*| *2025-04-16*|
|*Comments*| *Accepted for publication in MNRAS, 15 pages. FITS files are available at: this https URL*|
|**Abstract**|            The disk around PDS 70 hosts two directly imaged protoplanets in a gap. Previous VLT/SPHERE and recent James Webb Space Telescope/NIRCam observations have hinted at the presence of a third compact source in the same gap at ~13 au, interior to the orbit of PDS 70 b. We reduce seven published and one unpublished VLT/SPHERE datasets in YJH and K bands, as well as an archival VLT/NaCo dataset in L' band, and an archival VLT/SINFONI dataset in H+K band. We combine angular-, spectral- and reference star differential imaging to search for protoplanet candidates. We recover the compact source in all epochs, consistent with the JWST detection, moving on an arc that can be fit by Keplerian motion of a protoplanet which could be in a resonance with PDS 70 b & c. We find that the spectral slope is overall consistent with the unresolved star and inner disk emission at 0.95--1.65$\mu$m, which suggests a dust scattering dominated spectrum. An excess beyond 2.3$\mu$m could be thermal emission from either a protoplanet or heated circumplanetary dust, variability, or inner disk contamination, and requires confirmation. While we currently cannot rule out a moving inner disk feature or a dust clump associated with an unseen planet, the data supports the hypothesis of a third protoplanet in this remarkable system.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.11210-b31b1b.svg)](https://arxiv.org/abs/2504.11210) | **Differentiating Formation Models with New Dynamical Masses for the PDS 70 Protoplanets**  |
|| <mark>D. Trevascus</mark>, et al. -- incl., <mark>E. Matthews</mark>, <mark>I. Hammond</mark>, <mark>W. Brandner</mark>, <mark>R. Burn</mark>, <mark>G. Chauvin</mark>, <mark>L. Kreidberg</mark>, <mark>J. Sauter</mark> |
|*Appeared on*| *2025-04-16*|
|*Comments*| **|
|**Abstract**|            Hot- and cold-start planet formation models predict differing luminosities for the young, bright planets that direct imaging surveys are most sensitive to. However, precise mass estimates are required to distinguish between these models observationally. The presence of two directly imaged planets, PDS 70 b and c, in the PDS 70 protoplanetary disk provides us a unique opportunity for dynamical mass measurements, since the masses for these planets are currently poorly constrained. Fitting orbital parameters to new astrometry of these planets, taken with VLTI/GRAVITY in the $K$~band, we find $2\sigma$ dynamical upper mass limits of 4.9 $M_{\rm Jup}$ for b and 13.6 $M_{\rm Jup}$ for c. Adding astrometry from the newly proposed planet candidate PDS 70 d into our model, we determine $2\sigma$ dynamical upper mass limits of 5.3 $M_{\rm Jup}$, 7.5 $M_{\rm Jup}$ and 2.2 $M_{\rm Jup}$ for b, c, and the candidate d respectively. However, $N$-body analysis of the orbits fit in this case suggest that the inclusion of $d$ makes the system unstable. Using the upper mass limits for b and c we rule out the coldest-start formation models for both planets, calculating minimum post-formation entropies ($S_i$) of 9.5 $k_{\rm B}/{\rm baryon}$ and 8.4 $k_{\rm B}/{\rm baryon}$ respectively. This places PDS 70 b and c on the growing list of directly-imaged planets inconsistent with cold-start formation.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.11424-b31b1b.svg)](https://arxiv.org/abs/2504.11424) | **MINDS. Anatomy of a water-rich, inclined, brown dwarf disk: lack of abundant hydrocarbons**  |
|| <mark>G. Perotti</mark>, et al. -- incl., <mark>T. Henning</mark>, <mark>K. Schwarz</mark>, <mark>R. v. Boekel</mark> |
|*Appeared on*| *2025-04-16*|
|*Comments*| *30 pages, 18 figures, resubmitted the first revision to ApJ*|
|**Abstract**|            2MASS J04381486+2611399 (or J0438) is one of the few young brown dwarfs (BD) with a highly inclined ($i\!\sim\!70^\circ$) disk. Here we report results from JWST-MIRI MRS, HST-ACS and ALMA Band 7 observations. Despite its late spectral type (M7.25), the spectrum of J0438 resembles those of inner disks around earlier-type stars (K1-M5, T Tauri stars), with a volatile reservoir lacking hydrocarbons (except for acetylene, C$_2$H$_2$) and dominated by water. Other identified species are H$_2$, CO$_2$, HCN, [Ar$^{+}$], and [Ne$^{+}$]. The dominance of water over hydrocarbons is driven by multiple factors such as disk dynamics, young disk age, low accretion rate and possible inner disk clearing. J0438 appears highly dynamic, showing a seesaw-like variability and extended emission in H$_2 \,\,\, S$(1), $S$(3), $S$(5), [Ne$^{+}$] and CO ($J=3-2$). Interestingly, the CO emission reaches up to 400 au from the brown dwarf, suggesting ongoing infalling/outflowing activity impacting the disk chemistry. These observations underscore the combined power of JWST, HST and ALMA in characterizing the chemical diversity and dynamics of brown dwarf disks.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.11425-b31b1b.svg)](https://arxiv.org/abs/2504.11425) | **MINDS: The very low-mass star and brown dwarf sample -- Hidden water in carbon-dominated protoplanetary disks**  |
|| A. M. Arabhavi, et al. -- incl., <mark>T. Henning</mark>, <mark>G. Perotti</mark>, <mark>K. Schwarz</mark> |
|*Appeared on*| *2025-04-16*|
|*Comments*| *Accepted for publication in Astrophysical Journal Letters, 21 pages, 24 figures*|
|**Abstract**|            Infrared observations of the inner disks around very low-mass stars (VLMS, $<$0.3$\,M_{\odot}$) have revealed a carbon-rich gas composition in the terrestrial planet-forming regions. Contrary to the typically water-rich T Tauri disk spectra, only two disks around VLMS have been observed to be water-rich among more than ten VLMS disks observed so far with JWST/MIRI. In this letter, we systematically search for the presence of water and other oxygen-bearing molecules in the JWST/MIRI spectra of ten VLMS disks from the MIRI mid-INfrared Disk Survey (MINDS). In addition to the two previously reported detections of water emission in this VLMS sample, we detect water emission in the spectra of three other sources and tentatively in one source, and we provide strong evidence for water emission in the remaining disks in the MINDS sample, most of which have bright emission from carbon-bearing molecules. We show that the $\rm C_2H_2$ emission is much stronger than that of water for sources with low luminosities, and the hydrocarbons outshine the water emission in such conditions. We propose that the appearance of water-rich vs. hydrocarbon-rich spectra is related to the location of the water reservoir in the disk relative to the main hydrocarbon reservoir. Our findings indicate that the terrestrial planet forming regions in VLMS disks have high carbon-to-oxygen ratios (C/O$>$1), but can still harbor ample water similar to those in the T Tauri disks.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.10599-b31b1b.svg)](https://arxiv.org/abs/2504.10599) | **Dust continuum radiation maps from MHD simulations of accretion-ejection systems around single and binary stars**  |
|| S. Sheikhnezami, <mark>C. Fendt</mark>, S. Ataiee |
|*Appeared on*| *2025-04-16*|
|*Comments*| *27 pages,22 figures, Accepted for publication in Astrophysical Journal, comments are welcome*|
|**Abstract**|            We study the launching of magnetized jets from a resistive circumstellar disk within a binary system, employing a unique combination of 3D MHD jet launching simulations (PLUTO code) and post-processed 3D radiative transfer modeling (RADMC-3D code). Our findings reveal a well-defined jet originating from the inner region of the disk, extending to a larger disk area. While the model attains steady states for a single star, a binary system leads to the emergence of tidal effects such as the formation of ``spiral arms'' in the disk and inside the jet. Here we have consistently implemented a time-dependent Roche potential for the gravity of the binary. As a major step forward, we further present the first 3D radiation maps of the dust continuum for the disk-jet structure. In principle, this allows us to compare MHD simulation results to observed disk-outflow features. We, therefore, present convolved images of the dust continuum emission, employing exemplary point spread functions of the MIRI instrument (5~$\mu m$ band) and the ALMA array (320~$\mu m$ band). In these bands, we identify distinguishable features of the disk-jet structure, such as "spiral arms," which we have also seen in the MHD this http URL gas density increased by an order of magnitude, the disk become optically thick at 5~$\mu m$, but remains bright at 320~$\mu$m. At this wavelength, 320~$\mu$m, enhanced structural features in the disk and the base of the wind become more pronounced and are well resolved in the convolved image.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.10931-b31b1b.svg)](https://arxiv.org/abs/2504.10931) | **A Highly Efficient Cross-matching Scheme using Learned Index Structure**  |
|| P.-M. Lam, et al. -- incl., <mark>X. Zhang</mark>, <mark>Y. Wang</mark> |
|*Appeared on*| *2025-04-16*|
|*Comments*| **|
|**Abstract**|            Spatial data fusion is a bottleneck when it meets the scale of 10 billion records. Cross-matching celestial catalogs is just one example of this. To challenge this, we present a framework that enables efficient cross-matching using Learned Index Structures. Our approach involves a data transformation method to map multi-dimensional data into easily learnable distributions, coupled with a novel search algorithm that leverages the advantages of model pairs, significantly enhancing the efficiency of nearest-neighbor search. In this study, we utilized celestial catalog data derived from astronomical surveys to construct the index and evaluated the speed of the cross-matching process. Using the HEALPix segmentation scheme, we built an independent model object for each tile and developed an end-to-end pipeline to construct a framework with semantic guarantees for record retrieval in query and range search. Our results show that the proposed method improves cross-matching speed by more than four times compared to KD-trees for a radius range between 1 milli-arcseconds and 100 arcseconds.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2504.11127/./Figures/obs_big_4x3.png', 'tmp_2504.11127/./Figures/RDI_all_channels.png', 'tmp_2504.11127/./Figures/orbit_continuum_new.png']
copying  tmp_2504.11127/./Figures/obs_big_4x3.png to _build/html/
copying  tmp_2504.11127/./Figures/RDI_all_channels.png to _build/html/
copying  tmp_2504.11127/./Figures/orbit_continuum_new.png to _build/html/
exported in  _build/html/2504.11127.md
    + _build/html/tmp_2504.11127/./Figures/obs_big_4x3.png
    + _build/html/tmp_2504.11127/./Figures/RDI_all_channels.png
    + _build/html/tmp_2504.11127/./Figures/orbit_continuum_new.png
found figures ['tmp_2504.11210/./Entropy_bcd.png', 'tmp_2504.11210/./Entropy_bc.png', 'tmp_2504.11210/./Fig_SINFONI_PDS70c_NEGFC.png']
copying  tmp_2504.11210/./Entropy_bcd.png to _build/html/
copying  tmp_2504.11210/./Entropy_bc.png to _build/html/
copying  tmp_2504.11210/./Fig_SINFONI_PDS70c_NEGFC.png to _build/html/
exported in  _build/html/2504.11210.md
    + _build/html/tmp_2504.11210/./Entro

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\cp}[1]{\textcolor{red}{\textbf{CP:} #1}}$
$\newcommand{\vc}[1]{\textcolor{green}{\textbf{VC:} #1}}$
$\newcommand{\djp}[1]{\textcolor{magenta}{\textbf{DP:} #1}}$
$\newcommand{\ih}[1]{\textcolor{cyan}{\textbf{IH:} #1}}$
$\newcommand{\mum}{\ensuremath{\upmu\mathrm{m}}\xspace}$</div>



<div id="title">

# Keplerian motion of a compact source orbiting the inner disk of PDS 70: A third protoplanet in a resonance with b and c?

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2504.11127-b31b1b.svg)](https://arxiv.org/abs/2504.11127)<mark>Appeared on: 2025-04-16</mark> -  _Accepted for publication in MNRAS, 15 pages. FITS files are available at: this https URL_

</div>
<div id="authors">

<mark>I. Hammond</mark>, et al. -- incl., <mark>D. Trevascus</mark>, <mark>M. Samland</mark>

</div>
<div id="abstract">

**Abstract:** The disk around PDS 70 hosts two directly imaged protoplanets in a gap. Previous VLT/SPHERE and recent _James Webb Space Telescope_ /NIRCam observations have hinted at the presence of a third compact source in the same gap at $\sim$ 13 au, interior to the orbit of PDS 70 b.We reduce seven published and one unpublished VLT/SPHERE datasets in _YJH_ and _K_ bands, as well as an archival VLT/NaCo dataset in _L'_ band, and an archival VLT/SINFONI dataset in _H+K_ band. We combine angular-, spectral- and reference star differential imaging to search for protoplanet candidates.We recover the compact source in all epochs, consistent with the JWST detection, moving on an arc that can be fit by Keplerian motion of a protoplanet which could be in a resonance with PDS 70 b \& c. We find that the spectral slope is overall consistent with the unresolved star and inner disk emission at 0.95--1.65 $\mum$ , which suggests a dust scattering dominated spectrum. An excess beyond 2.3 $\mum$ could be thermal emission from either a protoplanet or heated circumplanetary dust, variability, or inner disk contamination, and requires confirmation. While we currently cannot rule out a moving inner disk feature or a dust clump associated with an unseen planet, the data supports the hypothesis of a third protoplanet in this remarkable system.

</div>

<div id="div_fig1">

<img src="tmp_2504.11127/./Figures/obs_big_4x3.png" alt="Fig5" width="100%"/>

**Figure 5. -** Nine years of high-contrast imaging data of PDS 70. All images are cropped to a 1$\farcs$2 $\times$ 1$\farcs$2 field of view centred on the star and in chronological order (left to right, top to bottom). The semi-transparent mask over A is 160 mas wide, roughly the inner working angle of the SPHERE coronagraph (SINFONI, NaCo and NIRCam are non-coronagraphic). The PSF-subtraction strategy is indicated in the top-left corner of the frame. The disk, b and candidate d are detected in all frames, whereas c is clearest at $\lambda>2 $\mum$$. North is up, East is to the left. Figure made using casa cube(\url{https://github.com/cpinte/casa_cube}).
     (*fig:observations*)

</div>
<div id="div_fig2">

<img src="tmp_2504.11127/./Figures/RDI_all_channels.png" alt="Fig9" width="100%"/>

**Figure 9. -** Iterative PCA-RDI for the 2022 Feb 28 epoch in Fig. \ref{fig:observations} truncated to seven principal components and obtained with three iterations, for each individual spectral channel. For all frames, the PSF of the reference star UCAC2 14412811 observed during the sequence has been subtracted. Candidate d is visible in all channels that are not contaminated by telluric absorption, whereas PDS 70 b is conspicuous at longer wavelengths (i.e., _H_-band). The grey patch represents the FWHM of the non-coronagraphic PSF and is unique for each channel. No numerical mask has been used to represent the location of the coronagraphic mask. Pixel intensities are on a linear scale with an intensity cut from the 0–99.5 percentiles of the entire cube. Figure made using hciplot(\url{https://github.com/carlos-gg/hciplot}). (*fig:all_channels*)

</div>
<div id="div_fig3">

<img src="tmp_2504.11127/./Figures/orbit_continuum_new.png" alt="Fig6" width="100%"/>

**Figure 6. -** Overview of the PDS 70 system, including 1000 random samples from the orbital posteriors of planets b (blue curves), c (yellow), and candidate d (green). The priors and median posterior for each orbital element retrieved with Octofitter are summarized in Table \ref{tab:priors}. A shift in declination of $-0$\farcs$02$ was applied to the continuum emission (colourscale) to enforce the c\textsubscript{smm} blob to overlap with near-IR astrometry of the same epoch. The grey beam of 44 $\times$ 37 mas with $\mathrm{PA} = 51\fdg5$ in the bottom-left is for the continuum emission originally presented in \citet{Benisty:2021uj}. (*fig:three_body_problem*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2504.11127"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\Mp}{\ensuremath{M_{\mathrm{p}}}\xspace}$
$\newcommand{\Sunits}{\ensuremath{k_{\mathrm{B}}/\mathrm{baryon}}\xspace}$
$\newcommand{\mum}{\ensuremath{\upmu}m\xspace}$
$\newcommand{\mas}{\ensuremath{\upmu}as\xspace}$
$\newcommand{\arraystretch}{1.4}$
$\newcommand{\arraystretch}{1.1}$</div>



<div id="title">

# Differentiating Formation Models with New Dynamical Masses for the PDS 70 Protoplanets

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2504.11210-b31b1b.svg)](https://arxiv.org/abs/2504.11210)<mark>Appeared on: 2025-04-16</mark> - 

</div>
<div id="authors">

<mark>D. Trevascus</mark>, et al. -- incl., <mark>E. Matthews</mark>, <mark>I. Hammond</mark>, <mark>W. Brandner</mark>, <mark>R. Burn</mark>, <mark>G. Chauvin</mark>, <mark>L. Kreidberg</mark>, <mark>J. Sauter</mark>

</div>
<div id="abstract">

**Abstract:** Hot- and cold-start planet formation models predict differing luminosities for the young, bright planets that direct imaging surveys are most sensitive to. However, precise mass estimates are required to distinguish between these models observationally. The presence of two directly imaged planets, PDS 70 _b_ and _c_ , in the PDS 70 protoplanetary disk provides us a unique opportunity for dynamical mass measurements, since the masses for these planets are currently poorly constrained. Fitting orbital parameters to new astrometry of these planets, taken with VLTI/GRAVITY in the $K$ band, we find $2\sigma$ dynamical upper mass limits of 4.9 $M_{\rm Jup}$ for _b_ and 13.6 $M_{\rm Jup}$ for _c_ . Adding astrometry from the newly proposed planet candidate PDS 70 _d_ into our model, we determine $2\sigma$ dynamical upper mass limits of 5.3 $M_{\rm Jup}$ , 7.5 $M_{\rm Jup}$ and 2.2 $M_{\rm Jup}$ for _b_ , _c_ , and the candidate _d_ respectively. However, $N$ -body analysis of the orbits fit in this case suggest that the inclusion of $d$ makes the system unstable. Using the upper mass limits for _b_ and _c_ we rule out the coldest-start formation models for both planets, calculating minimum post-formation entropies ( $S_i$ ) of 9.5 $\Sunits$ and 8.4 $\Sunits$ respectively. This places PDS 70 _b_ and _c_ on the growing list of directly-imaged planets inconsistent with cold-start formation.

</div>

<div id="div_fig1">

<img src="tmp_2504.11210/./Entropy_bcd.png" alt="Fig7" width="100%"/>

**Figure 7. -** Same as Figure \ref{fig:evo_comp_3body}, but with upper mass limits taken from the three-planet orbital parameter fit. (*fig:evo_comp_3body*)

</div>
<div id="div_fig2">

<img src="tmp_2504.11210/./Entropy_bc.png" alt="Fig6" width="100%"/>

**Figure 6. -** Comparison of the upper mass limits for PDS 70 _b_ and PDS 70 _c_ from the two-planet orbital parameter fit to cold-, warm-, and hot-start evolutionary models from \citet{Spiegel_2012}. The mass-luminosity curves for the different models, taken at the PDS 70 system age of 5.4 Myr are shown in the left panel, with their corresponding post-formation entropies given in the legend. The ribbon around each curve corresponds to the uncertainty in age of PDS 70. The blue and red boxes in the left panel show the possible masses and luminosities for _b_ and _c_ respectively, with luminosities taken from \citet{Wang_2020}. The upper right panel shows the luminosity-$S_i$ curve at the upper mass limit for _b_. The blue shaded area in this panel, bounded by the dashed and dashed-dotted lines shows the possible range of $S_i$ for _b_ from this orbital parameter fit. The lower right panel is the same as upper right panel, but for _c_. (*fig:evo_comp_2body*)

</div>
<div id="div_fig3">

<img src="tmp_2504.11210/./Fig_SINFONI_PDS70c_NEGFC.png" alt="Fig1" width="100%"/>

**Figure 1. -** Final image obtained from our new reduction of the 2014-05-10 SINFONI data, before (a) and after (b) subtracting the estimated contribution of planet _c_ as explained in Sec. \ref{sec:SINFONI}. (*fig:c_in_SINFONI*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2504.11210"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\angstrom}{\text{\normalfontÅ}}$
$\newcommand{\change}[1]{#1}$
$\newcommand{\arraystretch}{1.1}$
$\newcommand{\arraystretch}{1.1}$</div>



<div id="title">

# MINDS. Anatomy of a water-rich, inclined, brown dwarf disk: lack of abundant hydrocarbons

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2504.11424-b31b1b.svg)](https://arxiv.org/abs/2504.11424)<mark>Appeared on: 2025-04-16</mark> -  _30 pages, 18 figures, resubmitted the first revision to ApJ_

</div>
<div id="authors">

<mark>G. Perotti</mark>, et al. -- incl., <mark>T. Henning</mark>, <mark>K. Schwarz</mark>, <mark>R. v. Boekel</mark>

</div>
<div id="abstract">

**Abstract:** $\change{2MASS J04381486+2611399 (or J0438) is one of the few young brown dwarfs (BD) with a highly inclined ($i\!\sim\!70^\circ$) disk. Here we report results from JWST-MIRI MRS, HST-ACS and ALMA Band 7 observations. Despite its late spectral type (M7.25), the spectrum of J0438 resembles those of inner disks around earlier-type stars (K1–M5, T Tauri stars), with a volatile reservoir lacking hydrocarbons (except for acetylene, $C_2$$H_2$) and dominated by water. Other identified species are $H_2$, $CO_2$, HCN, [Ar$^{+}$], and [Ne$^{+}$]. The dominance of water over hydrocarbons is driven by multiple factors such as disk dynamics, young disk age, low accretion rate and possible inner disk clearing. J0438 appears highly dynamic, showing a seesaw-like variability and extended emission in $H_2     S$(1), $S$(3), $S$(5), [Ne$^{+}$] and CO ($J=3-2$). Interestingly, the CO  emission reaches up to 400 au from the brown dwarf, suggesting ongoing infalling/outflowing activity impacting the disk chemistry. These observations underscore the combined power of JWST, HST and ALMA in characterizing the chemical diversity and dynamics of brown dwarf disks.}$

</div>

<div id="div_fig1">

<img src="tmp_2504.11424/./J043814_BD_absorption.png" alt="Fig4" width="100%"/>

**Figure 4. -** **Absorption bands from the photosphere of the brown dwarf**. Comparison between the JWST-MIRI MRS spectrum of J0438 (black) and the PHOENIX model atmosphere spectrum (gold; \citealt{Husser2013}) described in Section \ref{app:stellar_contamination}. For illustration purposes, slab models highlighting the absorption bands of $H_2$O (light blue) and CO (light red) are also shown. (*fig:BD_absorption*)

</div>
<div id="div_fig2">

<img src="tmp_2504.11424/./J043814_mir_final.png" alt="Fig12" width="100%"/>

**Figure 12. -** **JWST-MIRI MRS spectrum of the young J0438 disk (black).** The spectrum is dominated by amorphous silicate absorption and emission bands typical of a highly inclined disk configuration. A very weak $CO_2$ ice absorption feature is tentatively detected (see Appendix \ref{app:cont_sub}, Fig. \ref{fig:co2_ice}) along with inner disk gas reservoir (Table \ref{tab:fluxes}) composed by molecular hydrogen ($H_2$), water ($H_2$O), carbon dioxide ($CO_2$), hydrogen cyanide (HCN), acetylene ($C_2$$H_2$) and the singly ionized cations of argon and neon ([Ar$^{+}$], [Ne$^{+}]$; for zoom-ins see Appendices \ref{app:stellar_contamination} and \ref{app:variability}). The comparison between the JWST-MIRI and the _Spitzer_-IRS low- (SL, LL) and high- (SH) resolution spectra reveal continuum and line variability (grey and teal). The _Spitzer_-IRS spectra were taken from CASSIS \citep{Lebouteiller2011,Lebouteiller2015} and \citet{Pascucci2013}. (*fig:full_spectra*)

</div>
<div id="div_fig3">

<img src="tmp_2504.11424/./J043814_nir_mir.png" alt="Fig3" width="100%"/>

**Figure 3. -** **Near- and mid-IR spectra of the young brown dwarf J0438.** Near-infrared IRTF (light blue; \citealt{Luhman2007}), _Spitzer_-IRAC photometry (blue points), _Spitzer_-IRS SL, LL (grey; PID: 40302; PI: J. R. Houck), _Spitzer_-IRS SH (teal; \citealt{Pascucci2013}) and JWST-MIRI MRS (dark blue; this work). The inset shows a zoom-in of the IRTF data. The prominent $H_2$O, CO and amorphous silicate absorption bands are labelled. (*fig:nir_mir*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2504.11424"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\ch}[1]{{\rm #1}}$
$\newcommand{\ama}[1]{\textcolor{red}{\rm #1^{(\rm AMA)}}}$
$\newcommand{\ik}[1]{\textcolor{purple}{\rm #1^{(\rm IK)}}}$
$\newcommand{\ikc}[1]{\textcolor{purple}{IK: \rm #1}}$
$\newcommand{\hl}[1]{{#1}}$
$\newcommand{\hll}[1]{{#1}}$
$\newcommand{\mybox}[2][1.1ex]{\raisebox{#1}{\myboxi{#2}}}$
$\newcommand{\myboxx}[2][1.1ex]{\raisebox{#1}{\myboxxi{#2}}}$
$\newcommand{\myboxxx}[2][1.1ex]{\raisebox{#1}{\myboxxxi{#2}}}$</div>



<div id="title">

# MINDS: The very low-mass star and brown dwarf sample \ Hidden water in carbon-dominated protoplanetary disks

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2504.11425-b31b1b.svg)](https://arxiv.org/abs/2504.11425)<mark>Appeared on: 2025-04-16</mark> -  _Accepted for publication in Astrophysical Journal Letters, 21 pages, 24 figures_

</div>
<div id="authors">

A. M. Arabhavi, et al. -- incl., <mark>T. Henning</mark>, <mark>G. Perotti</mark>, <mark>K. Schwarz</mark>

</div>
<div id="abstract">

**Abstract:** Infrared observations of the inner disks around very low-mass stars (VLMS, $<$ 0.3 $ M_{\odot}$ ) have revealed a carbon-rich gas composition in the terrestrial planet-forming regions. Contrary to the typically water-rich T Tauri disk spectra, only two disks around VLMS have been observed to be water-rich among more than ten VLMS disks observed so far with JWST/MIRI. In this letter, we systematically search for the presence of water and other oxygen-bearing molecules in the JWST/MIRI spectra of ten VLMS disks from the MIRI mid-INfrared Disk Survey (MINDS). In addition to the two previously reported detections of water emission in this VLMS sample, we detect water emission in the spectra of three other sources and tentatively in one source, and we provide strong evidence for water emission in the remaining disks in the MINDS sample, most of which have bright emission from carbon-bearing molecules. We show that the $\ch{C_2H_2}$ emission is much stronger than that of water for sources with low luminosities, and the hydrocarbons outshine the water emission in such conditions. $\hl{We propose that the appearance of}$ water-rich vs. hydrocarbon-rich spectra is related to the location of the water reservoir in the disk relative to the main hydrocarbon reservoir. Our findings indicate that the terrestrial planet forming regions in VLMS disks $\hl{have high carbon-to-oxygen ratios}$ (C/O $>$ 1), but can still harbor ample water similar to those in the T Tauri disks.

</div>

<div id="div_fig1">

<img src="tmp_2504.11425/./HiddenH2O_PPVII_TWA27_1.0.3.png" alt="Fig22" width="100%"/>

**Figure 22. -** Detections and non-detections of pure rotational line emission of water. The continuum subtracted spectra are shown in black and the water slab models in blue. The slab model parameters are arbitrarily chosen for a visual match with the observed spectra. The temperature and column densities of the models are the same across all the panels, while the emitting area is scaled to match the flux level. The vertical gray area in each panel shows the wavelength region used to calculate the noise level. The light gray shaded region across the spectra shows the $\pm$3$\sigma$ noise level obtained in this way. The identified water line transitions are indicated at the top of the figure as $J_{K_a K_c}-J'_{K_{a}' K_{c}'}$, where $J$, $K_a$, $K_c$ are the total angular momentum quantum number, and the projections of the angular momentum on the principal axes. (*fig:hiddenwater*)

</div>
<div id="div_fig2">

<img src="tmp_2504.11425/./water_corr_coeff_1.0.3.png" alt="Fig24" width="100%"/>

**Figure 24. -** Cross-correlations of observed VLMS spectra with water slab models between 16.65 and 17.65 $\mu$m for different column densities ($N$) and temperatures ($T$). The correlation coefficient varies from -1 (anti-correlation, in red) to +1 (correlation, in blue). The contours of correlation coefficients at 0.25, 0.5, and 0.75 are shown with dark gray lines. The parameter space where the statistical significance is low (i.e., the p-value is larger than 0.03) is hatched. The orange cross indicates the best correlation coefficient for each source. (*fig:water_corr*)

</div>
<div id="div_fig3">

<img src="tmp_2504.11425/./HiddenH2O_PPVII_TWA27_1.0.3_rovib.png" alt="Fig21" width="100%"/>

**Figure 21. -** Detections of ro-vibrational bands of water emission in J1605 (top), NC1 (middle), and J1558 (bottom). The continuum subtracted spectra are shown in black and the LTE water slab models are shown in blue. The slab model parameters (temperature, column density, and emitting radius) are chosen to roughly match the observed spectra and are not obtained from fits: 525 K, 10$^{20}$ cm$^{-2}$, 0.02 au for J1605; 525 K, 10$^{19}$ cm$^{-2}$, 0.02 au for NC1; 525 K, 10$^{18.33}$ cm$^{-2}$, 0.03 au for J1558. $\hl${The red dots show the detected water lines in the least contaminated window.} (*fig:hiddenwater_rovib*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2504.11425"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

443  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

12  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

5  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
